In [3]:
from pipeline import Pipeline
from pipeline.steps import *

pipeline_fe = Pipeline(
    steps=[
        LoadDataFrameStep("df_intermedio.parquet"),
        FilterProductForTestingStep(total_products_ids=1, random=False),

        CreateSerieIdStep(),
        DateRelatedFeaturesStep(),
        CastDataTypesStep(dtypes=
            {
                "product_id": "uint32", 
                "customer_id": "uint32",
                "mes": "uint16",
                "year": "uint16",
                "brand": "category",
                "cat1": "category",
                "cat2": "category",
                "cat3": "category",
            }
        ),

        ReduceMemoryUsageStep(),
        SaveDataFrameStep(df_name="df", file_name="df_tsfresh_base.pickle")
    ]
)


In [4]:
pipeline_fe = Pipeline(
    steps=[
        LoadDataFrameStep("df_intermedio.parquet"),
        FilterProductForTestingStep(total_products_ids=1, random=False),

    ]
)
pipeline_fe.run()

Executing step: LoadDataFrameStep
Step LoadDataFrameStep completed in 2.59 seconds
Executing step: FilterProductForTestingStep
Filtered DataFrame shape: (17862, 13)
Step FilterProductForTestingStep completed in 0.56 seconds


In [5]:
df = pipeline_fe.get_artifact("df")
df

,product_id,fecha,customer_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,stock_final,cat1,cat2,cat3,brand,sku_size
0,20001,2017-01,10001,0.0,11,99.438606,99.438606,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
1,20001,2017-02,10001,0.0,23,198.843643,198.843643,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
2,20001,2017-03,10001,0.0,33,92.465370,92.465370,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
3,20001,2017-04,10001,0.0,8,13.297280,13.297280,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
4,20001,2017-05,10001,0.0,15,101.207108,101.005630,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17857,20001,2017-01,10635,0.0,0,0.000000,0.000000,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
17858,20001,2017-02,10635,0.0,0,0.000000,0.000000,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
17859,20001,2017-03,10635,0.0,0,0.000000,0.000000,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
17860,20001,2017-02,10636,0.0,0,0.000000,0.000000,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0


In [ ]:
from tsfresh.utilities.dataframe_functions import roll_time_series
from tsfresh.feature_extraction import EfficientFCParameters, MinimalFCParameters

rolled_backward = roll_time_series(activity_data,
                                           column_id=id_column,
                                           column_sort=sort_column,
                                           column_kind=None,
                                           rolling_direction=1,
                                           max_timeshift=360)

TypeError: unsupported Type Index

In [32]:
model_test_pipeline = Pipeline(
    steps=[
        LoadDataFrameFromPickleStep(path="df_tsfresh_base.pickle"),
        SplitDataFrameStep(df="df", test_date="2019-10", gap=1),
        CreateTargetColumStep(target_col="tn"),
        #CreateMultiDiffTargetColumStep(target_col=tn),
        #CreateTargetColumDiffStep(target_col=tn),
        ScaleFeatureStep(column=".*tn.*", override=False, regex=True),  
        ScaleFeatureStep(column="target", override=True, scaler=PipelineMinMaxScaler),
        ReduceMemoryUsageStep(),
        PrepareXYStep(),
        #TrainModelStep(model_cls=XGBOOSTPipelineModel, params=xgb_params),
        #TrainModelStep(model_cls=MLPPipelineModel),
        TrainModelStep(folds=0, params={'num_leaves': 31, "n_estimators": 1200, "learning_rate": 0.01}),
        #TrainNNModelStep(model_cls=NNPipelineModel),
        #TrainModelStep(folds=0, params={'num_leaves': 31, "n_estimators": 200, "learning_rate": 0.1}),
        PredictStep(),
        #PredictNNModelStep(),
        InverseScalePredictionsStep(),
        IntegratePredictionsStep(),
        EvaluatePredictionsSteps(),
        PlotFeatureImportanceStep(),
        #SaveExperimentStep(exp_name="test_model",),
    ]
)
model_test_pipeline.run()

Executing step: LoadDataFrameFromPickleStep
Step LoadDataFrameFromPickleStep completed in 0.04 seconds
Executing step: SplitDataFrameStep


KeyError: 'fecha'